In [1]:
# import the libraries
import csv
import cv2
import numpy as np
print(cv2.__version__)

3.1.0


In [2]:
current_dir = "./180204-00/" #"./180203-00/"
model_name = "180209-00.h5"
IMG_SHAPE = (160, 320, 3)
lines = [] 
images = []
measurements = []

In [3]:
def add_training_data(directory):
    csv_fname = directory + "driving_log.csv"
    print(csv_fname)
    f_csv = open (csv_fname)
    print(f_csv)
    read_buffer = csv.reader(f_csv)
    for a_line in read_buffer:
        lines.append(a_line)

    #modify source path to relative path
    for a_line in lines:
        orig_source_path = a_line [0]  #center image

        #exclude any header lines
        if (a_line[0] == "center"):
            print(a_line)
            continue

        f_name = orig_source_path.split('\\')[-1]  # just the file name portion for windows based files
        current_image_path = directory +"IMG/" + f_name
        #print(current_image_path)
        an_img  = cv2.imread(current_image_path, cv2.IMREAD_COLOR)
        if (an_img == None):
            print("None file: ",current_image_path)
            continue
            
        #add the image & steering to the training set    
        images.append(an_img)
        measurements.append (float(a_line[3]))
        
        #add the flip side of the image
        #image_flipped = np.fliplr(image)
        #measurement_flipped = -measurement
        images.append(np.fliplr(an_img))
        measurements.append (-float(a_line[3]))
        
        #close the opened file
        f_csv.close()

In [4]:
add_training_data(current_dir)
#add_training_data("./180203-00/")

X_train = np.array(images)
y_train = np.array(measurements)
    
print("X_train shape {}, y_train.shape".format(X_train.shape, y_train.shape))

./180204-00/driving_log.csv
<_io.TextIOWrapper name='./180204-00/driving_log.csv' mode='r' encoding='UTF-8'>


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:23: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


X_train shape (27128, 160, 320, 3), y_train.shape


In [5]:
print(images[0])

[[[185 145 116]
  [185 145 116]
  [184 144 115]
  ..., 
  [ 53  66  64]
  [ 54  61  58]
  [ 70  75  73]]

 [[185 145 116]
  [185 145 116]
  [185 145 116]
  ..., 
  [ 20  43  39]
  [  9  31  26]
  [ 39  61  56]]

 [[186 146 117]
  [186 146 117]
  [186 146 117]
  ..., 
  [ 20  54  48]
  [ 15  53  47]
  [ 17  58  51]]

 ..., 
 [[ 71  81  81]
  [ 68  78  78]
  [ 64  74  74]
  ..., 
  [ 95 113 114]
  [ 93 111 112]
  [ 91 109 110]]

 [[ 45  55  55]
  [ 51  61  61]
  [ 58  68  68]
  ..., 
  [ 98 116 117]
  [ 94 112 113]
  [ 90 108 109]]

 [[ 43  53  53]
  [ 48  58  58]
  [ 53  63  63]
  ..., 
  [105 123 124]
  [ 94 112 113]
  [ 86 104 105]]]


In [6]:
N_EPOCHS = 10

In [ ]:
#Create and train the model.
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Dense, Flatten, Lambda
from keras.callbacks import ModelCheckpoint

In [ ]:
### Define your architecture here.
### TODO: Define your architecture.
krnl_sz = 3                ## kernel size for convolutions - smaller better for better localization.
pl_sz = 2                  ## pool size for max pooling
classes= 30                ## output classes - nornalized  x, y positions of each feature point x 15


#start with input layer with normalization of mean = 0 and range [-0.5: 0.5]
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape= IMG_SHAPE))


#Take NVIDIA paper suggested architecture as per 
# https://devblogs.nvidia.com/deep-learning-self-driving-cars/


## FIRST LAYER SET
##  Define the first 2D convolutional layer with proper input shape and 32 filters to start with, relu activation.
model.add(Conv2D(filters=32, kernel_size= krnl_sz, padding='same', activation='relu', 
                        input_shape=(96, 96, 1)))
## Add pooling layer
model.add(MaxPooling2D(pool_size= pl_sz))

## Add minimal drop out in early stage.
model.add(Dropout(0.10))    ## dropout reduces the risk of overfitting.


#### SECOND LAYER SET
## Define the second 2D with increased number of filters and relu activation.
model.add(Conv2D(filters=64, kernel_size=krnl_sz, padding='same', activation='relu'))

## Add pooling layer
model.add(MaxPooling2D(pool_size=pl_sz))

## Add moderate drop out to reduce overfitting.
model.add(Dropout(0.20))
          
####  THIRD LAYER SET
## Define the third 2D with increased number of filters and relu activation.
model.add(Conv2D(filters=128, kernel_size=krnl_sz, padding='same', activation='relu'))

## Add pooling layer
model.add(MaxPooling2D(pool_size=pl_sz))

## Add typical drop out to reduce overfitting.
model.add(Dropout(0.3))

# Flatten => RELU layers
model.add(Flatten())

###  Add Fully connected dense layer with 500 nodes
model.add(Dense(500, activation='relu'))

## Add strong drop out rate at this later stage.
model.add(Dropout(0.40))

### ensure default linear output without 'relu' or 'softmax'
model.add(Dense(classes)) #no softmax i.e. no non-linearity


# Summarize the model
model.summary()





In [7]:


model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape= IMG_SHAPE))
model.add(Flatten ())
model.add(Dense(1))

## TODO: Compile the model
model.compile(loss ='mse',optimizer='adam')
# TODO: Compile the model using a loss function and an optimizer.
#model.compile(loss = 'mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

## TODO: Train the model
checkpointer = ModelCheckpoint(filepath='weights.best.adam.hdf5', 
                               verbose=1, save_best_only=True)

# TODO: Run the model. Feel free to experiment with different batch sizes and number of epochs.
hist = model.fit(X_train, y_train, nb_epoch = N_EPOCHS, shuffle = True, callbacks=[checkpointer], verbose=1, validation_split=0.2)

#model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = N_EPOCHS)

#save model
model.save (model_name)


Using TensorFlow backend.


Train on 21702 samples, validate on 5426 samples
Epoch 1/10
21702/21702 [==============================] - 33s - loss: 4.0585 - val_loss: 2.8695
Epoch 2/10
21702/21702 [==============================] - 27s - loss: 3.8062 - val_loss: 11.0960
Epoch 3/10
21702/21702 [==============================] - 27s - loss: 4.4819 - val_loss: 3.2594
Epoch 4/10
21702/21702 [==============================] - 27s - loss: 3.9012 - val_loss: 5.9598
Epoch 5/10
21702/21702 [==============================] - 27s - loss: 4.0643 - val_loss: 3.8367
Epoch 6/10
21702/21702 [==============================] - 27s - loss: 4.3179 - val_loss: 6.1710
Epoch 7/10
21702/21702 [==============================] - 27s - loss: 4.1800 - val_loss: 2.8645
Epoch 8/10
21702/21702 [==============================] - 27s - loss: 4.6162 - val_loss: 3.3122
Epoch 9/10
21702/21702 [==============================] - 27s - loss: 3.8567 - val_loss: 5.9879
Epoch 10/10
21702/21702 [==============================] - 28s - loss: 4.2167 - val_lo